# Neurális háló építéséhez felhasznált program.

In [ ]:
import json
import joblib
import random
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, cross_validate

In [ ]:
testLocations = [
    "./parametertestResults/sampleprogram.json",
    "./parametertestResults/ebizzy.json",
    "./parametertestResults/ctx-clock.json",
    "./parametertestResults/fs_mark.json",
    "./parametertestResults/stream.json",
    "./parametertestResults/glmark2.json"
]
testNameList = [
    "sampleprogram",
    "ebizzy",
    "ctx-clock",
    "fs_mark",
    "stream",
    "glmark2"
]

In [ ]:
def avgLst(lst):
    if(isinstance(lst,list)):
        return sum(lst)/len(lst)
    else:
        return lst

def getIntervals(data):
    intervalTop,intervalBottom = 0.0,float(avgLst(data["measurements"][0]["results"]))
    for j in data["measurements"]:
        if(float(avgLst(j["results"])) > intervalTop):
            intervalTop = float(avgLst(j["results"]))
        if(float(avgLst(j["results"])) < intervalBottom):
            intervalBottom = float(avgLst(j["results"]))
    minMaxLst = [intervalBottom,intervalTop]
    return minMaxLst

def featureScale(value,intervalMax,intervalMin):
    return (value-intervalMin)/(intervalMax-intervalMin)

def getPredictionDf(predictions_lst,tType,df):
    return df[(df["testType"]==tType) & (df["latency"]==predictions[0][0]) &(df["min_gran"]==predictions[0][1]) &(df["wakeup_gran"]==predictions[0][2]) &(df["priority"]==predictions[0][3]) & (df["vm.swappiness"]==predictions[0][4]) ]

In [ ]:
classResultIntervals = [0.20,0.4,0.6,0.8]
classIntervalMinMax = []
dataStrLst = []
for i in range(len(testLocations)):
    file = open(testLocations[i])
    data = json.load(file)
    dataStrLst.append(json.dumps(data))
    minMax = getIntervals(data)
    if(i == 0 or i == 2):
        minMax.reverse()
    classIntervalMinMax.append(minMax)

In [ ]:
dataSet = []
for i in range(len(dataStrLst)):
    data_json = json.loads(dataStrLst[i])
    classResult = 0
    for j in data_json["measurements"]:
        swap = 1 if ( int(j["parameters"]["vm.swap"]) > 0 ) else 0                                     
        testType = data_json["testType"]
        testName = data_json["testName"]
        batchProcess = 1 if ( int(j["parameters"]["prio"]) > 0 ) else 0
        serverWorkload = 1 if( int(j["parameters"]["min_gran"]) > 1500000) else 0
        resultNormalized = featureScale(float(avgLst(j["results"])),classIntervalMinMax[i][1],classIntervalMinMax[i][0])
        for k in range(len(classResultIntervals)):
            if(resultNormalized > classResultIntervals[k]):
                classResult = (k+1)
        dataSet.append([testType,testName,batchProcess,classResult,serverWorkload,swap,resultNormalized,int(j["parameters"]["latency"]),int(j["parameters"]["min_gran"]),int(j["parameters"]["wakeup_gran"]),int(j["parameters"]["prio"]),int(j["parameters"]["vm.swap"])],)
df = pd.DataFrame(dataSet,columns=["testType","testName","batchProcess","resultClass","serverWorkload","swap","results","latency","min_gran","wakeup_gran","priority","vm.swappiness"])

In [ ]:
testTypeMap = {
    "cpu-s":0,
    "cpu" : 1,
    "system" : 2,
    "disk" : 3,
    "memory" : 4,
    "graphics":5
}
df.testType.replace(to_replace=testTypeMap,inplace=True)
#features
X = df[["testType","resultClass","batchProcess","serverWorkload","swap"]]
#labels
y = df[["latency","min_gran","wakeup_gran","priority","vm.swappiness"]]
#split dataset 80% train 20% test

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)


In [ ]:
#végig próbálja ezekkel a beállításokkal betanítani a modelt
params = {
            'estimator__activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'estimator__solver' : ['lbfgs', 'sgd', 'adam'],
            'estimator__hidden_layer_sizes': [(1,),(2,),(3,),(4,),(5,),(6,),(7,),(8,),(9,),(10,),(11,), (12,),(13,),(14,),(15,),(16,),(17,),(18,),(19,),(20,),(21,)]
}

model = GridSearchCV( MultiOutputClassifier(MLPClassifier(random_state=0,max_iter=10000000)) ,param_grid=params)
model.fit(X_train,y_train)
#végül kíirja a legjobb paraméterkészletet
print("A fejlesztési készleten található legjobb paraméterkészlet:")
print(model.best_params_)

Program kimenete: 
{'estimator__activation': 'tanh', 'estimator__hidden_layer_sizes': (9,), 'estimator__solver': 'lbfgs'}


Ezeket fogom felhasználni a model építéséhez. Sajnos nincs mágikus képlet a rejtett rétegek neuronjainak optimális számának kiválasztására. Létezik néhány "thumb rule" a neuronok számainak meghatározásához.
Találtam egy szabályt, miszerint a felső korlát meghatározására használható a következő képlet:
\begin{equation}
N_h = \frac{N_s} {(\alpha * (N_i + N_o))}
\end{equation}
Ahol 
* $N_i$  = az input neuronok száma.
* $N_o$ = az output neuronok száma.
* $N_s$ = a tanító minták száma
* $\alpha$ = egy tetszőleges méretezési tényező, általában 2-10.

Egy durva közelítés a geometriai piramisszabály segítségével nyerhető. Egy három rétegű hálózat esetén, ahol $n$ a bemenetek száma és $m$ a kimeneti elemek számát jelöli, a szabály így néz ki: $\sqrt{n*m}$.
A program nekem 9 neuront választott a legjobbnak, ez a piramisszabályhoz viszonylag közeli érték.